In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
from PIL import Image

# Check if CUDA is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Define the transformation for the input image
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
data_dir = r'C:\PlayGround\Automated-Blood-Group-Detection-main\Automated-Blood-Group-Detection-main\dataset_blood_group'
image_dataset = datasets.ImageFolder(data_dir, transform=data_transform)

# Split the dataset into training and validation sets (80/20 split)
train_size = int(0.8 * len(image_dataset))
val_size = len(image_dataset) - train_size
train_dataset, val_dataset = random_split(image_dataset, [train_size, val_size])

# Optimize data loading
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

dataloaders = {'train': train_loader, 'val': val_loader}

# Define a simpler CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 8)  # 8 classes for the 8 blood groups

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
num_epochs = 25
for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            print("Training...")
        else:
            model.eval()
            print("Validating...")

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            print(f"Batch done: Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'fingerprint_blood_group_model.pth')


Using device: cpu

Epoch 1/25
Training...
Batch done: Loss: 2.1095
Batch done: Loss: 1.9917
Batch done: Loss: 2.1994
Batch done: Loss: 2.3189
Batch done: Loss: 2.0805
Batch done: Loss: 2.0324
Batch done: Loss: 2.0516
Batch done: Loss: 2.1378
Batch done: Loss: 2.0281
Batch done: Loss: 2.1311
Batch done: Loss: 2.1090
Batch done: Loss: 1.9230
Batch done: Loss: 2.0743
Batch done: Loss: 1.9653
Batch done: Loss: 1.8479
Batch done: Loss: 2.1206
Batch done: Loss: 2.0127
Batch done: Loss: 1.8322
Batch done: Loss: 1.8966
Batch done: Loss: 2.0774
Batch done: Loss: 1.7087
Batch done: Loss: 1.7812
Batch done: Loss: 1.7732
Batch done: Loss: 1.7113
Batch done: Loss: 1.8844
Batch done: Loss: 1.7142
Batch done: Loss: 1.6438
Batch done: Loss: 1.4828
Batch done: Loss: 1.7930
Batch done: Loss: 1.6023
Batch done: Loss: 1.6942
Batch done: Loss: 1.7954
Batch done: Loss: 1.5615
Batch done: Loss: 1.5204
Batch done: Loss: 1.6320
Batch done: Loss: 1.7663
Batch done: Loss: 1.5521
Batch done: Loss: 1.4706
Batch do

In [3]:
from tkinter import Tk, filedialog, messagebox

In [14]:
# Prediction function
def predict_image(image_path, model):
    image = Image.open(image_path).convert('RGB')
    image = data_transform(image)
    image = image.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
    
    return preds.item()

# Function to upload image and predict blood group
def upload_and_predict():
    root = Tk()
    root.withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.bmp;*.jpg;*.jpeg;*.png")])
    
    if not file_path:
        messagebox.showinfo("Info", "No file selected")
        return
    
    predicted_class = predict_image(file_path, model)
    blood_groups = ['A-', 'A+', 'AB-', 'AB+', 'B-', 'B+', 'O-', 'O+']
    result = f'The predicted blood group is: {blood_groups[predicted_class]}'
    messagebox.showinfo("Prediction Result", result)

# Call the upload and predict function
upload_and_predict()